# StepFunctions Data Science SDK for BYO Container

Here we demonstrate how to use the StepFunction Data Science SDK to do an end-to-end data science workflow where you bring your own code and deploy a model on SageMaker.

The Steps are as follows:


1/ Create a Lambda function which launches a CodeBuild job that launches the creation of your Docker container. (Steps for this are included separately)

2/ Launch the Lambda function as a Step Functions workflow. 

3/ Once the Docker container is built, launch a SageMaker training job using SF DS SDK.

4/ Use the DS SDK to deploy the trained model.

## TODO: IAM Roles and Permissions

Before running the code, ensure that your Amazon SageMaker notebook IAM role can call the AWS StepFunctions SDK, and vice-versa.

To do this, follow the steps in the Setup Section of this notebook upto the section "Configure Execution Roles": https://github.com/awslabs/amazon-sagemaker-examples/blob/master/step-functions-data-science-sdk/machine_learning_workflow_abalone/machine_learning_workflow_abalone.ipynb


If you have already completed this, then ignore this section and move on to the next.

## Import necessary libraries

In [101]:
# Import required libraries and upload the training data to S3
import boto3
import os
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
import time
import uuid



role = get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket() # or feel free to replace with a bucket of your choosing
WORK_DIRECTORY = 'PennFudanPed'
key = 'BYO-Mask-RCNN'
prefix = '{}/{}'.format(key, WORK_DIRECTORY)

In [2]:
# install StepFunctions SDK
import sys
!{sys.executable} -m pip install --upgrade stepfunctions

Requirement already up-to-date: stepfunctions in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (1.0.0.4)
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import stepfunctions
import logging
from stepfunctions.steps import (LambdaStep, Retry, Catch, Fail, Chain, TrainingStep, ModelStep, EndpointConfigStep, EndpointStep)
from stepfunctions.workflow import Workflow
from stepfunctions.template.pipeline import TrainingPipeline
from stepfunctions.inputs import ExecutionInput

stepfunctions.set_stream_logger(level=logging.INFO)

# TODO: replace the IAM role below with the StepFunctionsWorkflowExecutionRole ARN from above in "<>"
workflow_execution_role = "arn:aws:iam::{accountnum}:role/StepFunctionsSageMakerRole" 

## Upload the training dataset to Amazon S3

In [4]:
data_location = sess.upload_data(WORK_DIRECTORY, bucket=bucket, key_prefix=prefix)
print(data_location)

s3://sagemaker-us-east-1-389535300735/BYO-Mask-RCNN/PennFudanPed


# Define Estimator and StepFunctions Workflow

In [43]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Note that this image name will work below provided you have made the changes to the Environment variables in the Lambda
#defintion as suggested in the workshop Readme. If not please make those first.

image = '{}.dkr.ecr.{}.amazonaws.com/sm-container-maskrcnn:torch'.format(account, region) 

maskrcnn = sagemaker.estimator.Estimator(image,
                       role, 1, 'ml.p2.xlarge', #feel free to modify with your own. A cost estimate is provided in Readme.
                       output_path="s3://{}/{}/output".format(sess.default_bucket(), key),
                       sagemaker_session=sess)

maskrcnn.set_hyperparameters(num_epochs = 1,
                              num_classes = 2)

#maskrcnn.fit(os.path.dirname(data_location))

### Create StepFunction Pipeline

In [44]:
lambda_state = LambdaStep(
    state_id="Calls CodeBuild to Build Container",
    parameters={  
        "FunctionName": "Docker_Lambda", #TODO: replace with the name of the Lambda function you created
        "Payload": {  
           "input": "HelloWorld"
        }
    }
)

lambda_state.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=15,
    max_attempts=2,
    backoff_rate=4.0
))

lambda_state.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("LambdaTaskFailed")
))

In [87]:
execution_input = ExecutionInput(schema={
    'JobName': str, 
    'ModelName': str,
    'EndpointName': str
})

In [88]:
train_step = TrainingStep(
    'Train Step', 
    estimator=maskrcnn,
#    role=workflow_execution_role,
    data=os.path.dirname(data_location),
    job_name=execution_input['JobName']
)

In [89]:
model_step = ModelStep(
    'Save model',
    model=train_step.get_expected_model(),
    model_name=execution_input['ModelName'] 
)

In [90]:
endpoint_config_step = EndpointConfigStep(
    "Create Endpoint Config",
    endpoint_config_name=execution_input['ModelName'],
    model_name=execution_input['ModelName'],
    initial_instance_count=1,
    instance_type='ml.m5.large'
)

In [91]:
endpoint_step = EndpointStep(
    "Create Endpoint",
    endpoint_name=execution_input['EndpointName'],
    endpoint_config_name=execution_input['ModelName']
)

In [92]:
workflow_definition = Chain([
    lambda_state,
    train_step,
    model_step,
    endpoint_config_step,
    endpoint_step
])

# Next, we define the workflow
workflow = Workflow(
    name="MyWorkflow-BYOC-MaskRCNN-{}".format(uuid.uuid1().hex),
    definition=workflow_definition,
    role=workflow_execution_role
)

In [93]:
print(workflow.definition.to_json(pretty=True))

{
    "StartAt": "Calls CodeBuild to Build Container",
    "States": {
        "Calls CodeBuild to Build Container": {
            "Parameters": {
                "FunctionName": "Docker_Lambda",
                "Payload": {
                    "input": "HelloWorld"
                }
            },
            "Resource": "arn:aws:states:::lambda:invoke",
            "Type": "Task",
            "Next": "Train Step",
            "Retry": [
                {
                    "ErrorEquals": [
                        "States.TaskFailed"
                    ],
                    "IntervalSeconds": 15,
                    "MaxAttempts": 2,
                    "BackoffRate": 4.0
                }
            ],
            "Catch": [
                {
                    "ErrorEquals": [
                        "States.TaskFailed"
                    ],
                    "Next": "LambdaTaskFailed"
                }
            ]
        },
        "Train Step": {
            "Resource":

In [94]:
workflow.render_graph()

In [95]:
workflow.create()

[INFO] Workflow created successfully on AWS Step Functions.


'arn:aws:states:us-east-1:389535300735:stateMachine:MyWorkflow-BYOC-MaskRCNN-76b9b6dc445c11ea8176cdf90f0dd667'

In [96]:
execution = workflow.execute(
    inputs={
        'JobName': 'BYOC-Mask-RCNN-{}'.format(uuid.uuid1().hex), # Each Sagemaker Job requires a unique name
        'ModelName': 'BYOC-Mask-RCNN-{}'.format(uuid.uuid1().hex), # Each Model requires a unique name,
        'EndpointName': 'BYOC-Mask-RCNN-{}'.format(uuid.uuid1().hex) # Each Endpoint requires a unique name,
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


## Watch the progress of your workflow here

In [ ]:
from IPython.display import display, display_html
while True:
    display_html(execution.render_progress())
    time.sleep(60)

Success 
 
 
 
 Failed 
 
 
 
 Cancelled 
 
 
 
 In Progress 
 
 
 
 Caught Error 
 
 
 

 
 Inspect in AWS Step Functions

## Inferences

Once your model is deployed, you can run inferences using this endpoint by using the SageMaker RealTimePredictor API. Please refer to the existing SageMaker documentation for how to do this.

Also to ensure you don't rack up costs, make sure you delete the endpoint once you are done. 

## End